In [1]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tvillight/l_hchat"
prompt = "I have fever and sore throat. What should I do?"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    f'[INST] {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:147

Result: [INST] I have fever and sore throat. What should I do? [/INST]  I'm sorry to hear that you have fever and sore throat. It could be a sign of an infection. Here are some things you can try to help manage your symptoms: 

1. Rest: Rest is necessary, especially if you have a viral infection that causes fever. Aim to get at least 8 hours of sleep each night and take naps during the day. 
2. Hydration: To help prevent dehydration, consume plenty of fluids like water, clear broths, or electrolyte-rich beverages like sports drinks. 
3. Pain relief: For throat pain, you can take acetaminophen (Tylenol) or ibuprofen (Advil or Motrin). Always follow the recommended dosage and


In [3]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [7]:
prompt="I have terrible knee pain and backache. What could be the reason?"

In [8]:
sequences = pipeline(
    f'[INST] {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST] I have terrible knee pain and backache. What could be the reason? [/INST]  There are several potential reasons for knee pain and backache. Here are some possible causes:

Knee Pain:

1. Osteoarthritis: This is one of the most common causes of knee pain. It occurs when the cartilage in the joint wears down over time, leading to inflammation and pain.
2. Meniscal tears: The meniscus is a cartilage structure in the knee joint that can tear due to injury or overuse. This can cause knee pain, especially when bending or twisting.
3. Ligament sprains: Sprains in the ligaments surrounding the knee joint can cause pain, especially when moving the knee.
4. Tendinitis: Inflammation of the tendons, especially the quadriceps or hamstring tendons, can cause knee pain.

Backache:

1. Herniated disc: When the gel-like center of a spinal disc bulges out through a tear in the outer layer, it can put pressure on nearby nerves and cause back pain.
2. Muscle strain: Overuse or injury to the 